In [32]:
import sympy as sp
import numpy as np 
from numpy import radians
from time import perf_counter

In [33]:
num_poles = 2

pole_as = [0.1 for i in range(num_poles)]
pole_ls = [0.2 for i in range(num_poles)]
pole_ms = [0.05 for i in range(num_poles)]
pole_ds = [0.01 for i in range(num_poles)]
pole_Js = [0.1 for i in range(num_poles)]

m_c = 0.5
g = 9.81

In [34]:
t = sp.Symbol("t")
s = sp.Function("s")(t)
d_s = sp.diff(s, t)
dd_s = sp.diff(d_s, t)
thetas = [sp.Function(f"theta_{i+1}")(t) for i in range(num_poles)]
d_thetas = [sp.diff(theta,t) for theta in thetas]
dd_thetas = [sp.diff(d_theta,t) for d_theta in d_thetas]
tau = sp.Symbol("tau")

In [35]:
pole_pcs = []
for i, (theta, a) in enumerate(zip(thetas, pole_as)):
    prev_1 = 0
    prev_2 = 0
    for prev_l, prev_theta in list(zip(pole_ls, thetas))[:i]:
        prev_1 += -prev_l*sp.sin(prev_theta)
        prev_2 += prev_l*sp.cos(prev_theta)
    pole_pcs.append([s-a*sp.sin(theta)+prev_1, a*sp.cos(theta)+prev_2])

In [36]:
def square_sum(x: tuple):
    result = 0
    for x_i in x:
        result += x_i**2
    return result

T = [1/2*m_c*d_s**2]
for m, pc, J, d_theta in zip(pole_ms, pole_pcs, pole_Js, d_thetas):
    d_pc = [sp.diff(p,t) for p in pc]
    T_p = 1/2*m*square_sum(d_pc) + 1/2*J*d_theta**2
    T.append(T_p)
    print(T_p)
    
V = 0   
for m, pc in zip(pole_ms, pole_pcs  ):
    V += g*m*pc[1]
R = 0
prev_w = 0
for d, d_theta in zip(pole_ds, d_thetas):
    R += 1/2*d*(d_theta-prev_w)**2
    prev_w = d_theta

0.025*(-0.1*cos(theta_1(t))*Derivative(theta_1(t), t) + Derivative(s(t), t))**2 + 0.00025*sin(theta_1(t))**2*Derivative(theta_1(t), t)**2 + 0.05*Derivative(theta_1(t), t)**2
0.001*(-sin(theta_1(t))*Derivative(theta_1(t), t) - 0.5*sin(theta_2(t))*Derivative(theta_2(t), t))**2 + 0.025*(-0.2*cos(theta_1(t))*Derivative(theta_1(t), t) - 0.1*cos(theta_2(t))*Derivative(theta_2(t), t) + Derivative(s(t), t))**2 + 0.05*Derivative(theta_2(t), t)**2


In [37]:
L = T[0]-V
eqs = []
lh = sp.diff(sp.diff(L, d_s),t) - sp.diff(L, s) + sp.diff(R, d_s)
rh = tau
eqs.append(lh-rh)
for theta, d_theta, T_p in zip(thetas, d_thetas, T[1:]):
    L = T_p-V
    lh = sp.diff(sp.diff(L, d_theta),t) - sp.diff(L, theta) + sp.diff(R, d_theta)
    rh = 0
    eqs.append(lh-rh)

In [38]:
sols = sp.solve(eqs, dd_thetas + [tau])
for key, value in sols.items():
    sols[key] = sp.simplify(value)
    display(sols[key])

0.5*Derivative(s(t), (t, 2))

1.46417910447761*sin(theta_1(t)) + 0.0497512437810945*cos(theta_1(t))*Derivative(s(t), (t, 2)) - 0.199004975124378*Derivative(theta_1(t), t) + 0.0995024875621891*Derivative(theta_2(t), t)

0.00995024875621891*sin(theta_1(t) - theta_2(t))*Derivative(theta_1(t), t)**2 + 0.00049503725155318*sin(theta_1(t))**2*cos(theta_2(t))*Derivative(s(t), (t, 2)) - 0.00049503725155318*sin(theta_1(t))*sin(theta_2(t))*cos(theta_1(t))*Derivative(s(t), (t, 2)) - 0.0145689463132101*sin(theta_1(t))*cos(theta_1(t) - theta_2(t)) + 0.488059701492537*sin(theta_2(t)) + 0.00198014900621272*cos(theta_1(t) - theta_2(t))*Derivative(theta_1(t), t) - 0.000990074503106359*cos(theta_1(t) - theta_2(t))*Derivative(theta_2(t), t) + 0.0492562065295414*cos(theta_2(t))*Derivative(s(t), (t, 2)) + 0.0995024875621891*Derivative(theta_1(t), t) - 0.0995024875621891*Derivative(theta_2(t), t)

In [39]:
str(sols[dd_thetas[0]])

'1.46417910447761*sin(theta_1(t)) + 0.0497512437810945*cos(theta_1(t))*Derivative(s(t), (t, 2)) - 0.199004975124378*Derivative(theta_1(t), t) + 0.0995024875621891*Derivative(theta_2(t), t)'

In [40]:
sp.sympify(str(sols[dd_thetas[0]]))

1.46417910447761*sin(theta_1(t)) + 0.0497512437810945*cos(theta_1(t))*Derivative(s(t), (t, 2)) - 0.199004975124378*Derivative(theta_1(t), t) + 0.0995024875621891*Derivative(theta_2(t), t)

In [44]:
def sympy2casadi(sympy_expr,sympy_var,casadi_var):
  import casadi
  assert casadi_var.is_vector()
  if casadi_var.shape[1]>1:
    casadi_var = casadi_var.T
  casadi_var = casadi.vertsplit(casadi_var)
  from sympy.utilities.lambdify import lambdify

  mapping = {'ImmutableDenseMatrix': casadi.blockcat,
             'MutableDenseMatrix': casadi.blockcat,
             'Abs':casadi.fabs
            }
  f = lambdify(sympy_var,sympy_expr,modules=[mapping, casadi])
  return f(*casadi_var)
 
import casadi as ca

sp_vars = [s,d_s,dd_s] + thetas + d_thetas + dd_thetas
xy = sp.Matrix(sp_vars)

ca_vars= [ca.SX.sym("s"), ca.SX.sym("d_s"), ca.SX.sym("dd_s")]
for i in range(1,1+num_poles):
  ca_vars.append(ca.SX.sym("theta"+str(i)))
  ca_vars.append(ca.SX.sym("d_theta"+str(i)))
  ca_vars.append(ca.SX.sym("dd_theta"+str(i)))
XY = ca.vertcat(*ca_vars)

print(sympy2casadi(sols[dd_thetas[0]],xy,XY))

(((((0.0497512*dd_s)*cos(theta0))+(0.0995025*theta1))-(0.199005*dd_theta0))+(1.46418*sin(theta0)))


In [ ]:
values = np.array([10, 0, radians(0), 0, radians(0), 0, radians(0), 0] + [0])
variables = [s, d_s]
for theta, d_theta in zip(thetas, d_thetas):
    variables.append(theta)
    variables.append(d_theta)
variables.append(u)

start_time = perf_counter()
# substitute the values of the variables into the solutions
sol_sub = sols.copy()
for i in range(100):
    for k, v in sols.items():
        a = v.subs({var: value for var, value in zip(variables, values)})
print(f"Time: {perf_counter()-start_time}")

Time: 0.2281134999357164
